# Viscoelastic multiple synthetic analyses template

This notebook provides a launchpad for testing *DeepMoD's* capabilities under a specified range of parameters for the problem of linear viscoelasticity and the Generalized Differential Model (GDM). The analysis is entirely based on the analysis fully explained in the notebook 'Example - VE - Synthetic Data Generation'. Here the focus is also on synthetic scenarios, where a manipulation is defined, the response simulated, and DeepMoD applied as an analysis tool to the results. The structure of this notebook is:

1. 1 or more parameters are chosen to be varied, and the values to be scanned through are defined. The 'parameters' may be anything, from the definition of the manipulation, to the scaling applied to the data series, to the hyper-parameters of the optimization process.

2. Given the above varying parameters, all preperation common to every test being conducted is worked through.

3. Any parts of the process that is affected by the varied paramters is embedded into a loop, and this loop is run to perform teh full analysis as many times as necessary to work through all the values of the parameters being varied.

Depending on the parameters varied, the relative sizes of steps 2 and 3 can vary hugely. In the default state of this notebook, **ALL** processing will be contained in step 3, with step 2 left blank and can be expanded at the discretion of future users. The exception to this is the definition of the library function which is core to the problem type, and not intended to be varied.

No attempt will be made in this notebook to guide the reader through any parts that exist also in the notebook 'Example - VE - Synthetic Data Generation' however explanation will be made for any novel parts.

In [ ]:
import os
import sys
from datetime import datetime
import pickle
import numpy as np
import torch

sys.path.append('../src')
import deepymod_torch.VE_datagen as VE_datagen
import deepymod_torch.VE_params as VE_params
from deepymod_torch.DeepMod import run_deepmod

## Setting varied parameters

Here we define the values we wish to test. This must be defined as an iterable of some kind. For testing, the geometric progression generator from NumPy is recommended. The switch between these values will occur at the top of the loop for each test, and the integration of the value will depend on the value being tested.

In [ ]:
investigation_title = 'Synthetic Noise - solid paper example'

In [ ]:
param_vals_1 = np.geomspace(np.sqrt(10)/100, 0.1, num=4)[-2:0:-1]
param_vals_1

## Common processing and configuration

[Move as much code here as possible to decrease computational cost. Any parts of the process unaffected by the varying parameters can be placed here.]

In [ ]:
import torch.autograd as auto
    
def mech_library(inputs, **library_config):
    '''
    Library function for the problem of linear viscoelasticity. Calculates all derivatives up to a specified order for strain and stress.
    The first derivative of strain is returned as the mandatory guiding side of the discovered model.
    All terms calculated from strain in the returned library are multiplied by -1.
    
    Parameters
        inputs: 2-tuple
            A tuple of (prediction, data) where both are Nx1 tensors and gradients have been tracked from data to prediction.
            The tensor, data, is the input to a neural network, whereas prediction is the output.
        **library_config: kwargs packed into dict.
            All additional configuration options. Mandatory is the inclusion of diff_order=int, input_type=string, and input_theta=2D tensor.
            input_type must be either 'Strain' or 'Stress'
            input_theta contains the library terms for the manipulation variable, including the first strain derivative if strain is the manipulation variable.
            input_theta is therefore Nx(diff_order+1) always.
            
    Returns
        [strain_t]: list
            Contains a Nx1 tensor
        theta: Nx(2*diff_order + 1) tensor
    '''
    
    prediction, data = inputs
    
    # Load already calculated derivatives of manipulation variable
    input_theta = library_config['input_theta']
    if data.shape[0] == 1: # Swaps real input_theta out for dummy in initialisation pass.
        input_theta = torch.ones((1, input_theta.shape[1]))
    
    # Automatic derivatives of response variable 
    output_derivs = auto_deriv(data, prediction, library_config['diff_order'])
    output_theta = torch.cat((prediction, output_derivs), dim=1)
    
    # Identify the manipulation/response as Stress/Strain and organise into returned variables
    if library_config['input_type'] == 'Strain':
        strain = input_theta
        stress = output_theta
    else: # 'Stress'
        strain = output_theta
        stress = input_theta
        
    strain_t = strain[:, 1:2] # Extract the first time derivative of strain
    strain = torch.cat((strain[:, 0:1], strain[:, 2:]), dim=1) # remove this before it gets put into theta
    strain *= -1 # The negatives of all strain terms are included in the library so that all coefficients discovered should be positive.
    theta = torch.cat((strain, stress), dim=1)
    
    return [strain_t], theta


def auto_deriv(data, prediction, max_order):
    '''
    Computes all derivatives up to a specified order using automatic differentiation for a single input and a single output of a neural network.
    If it is desired to calculate the derivatives of different predictions wrt different data, this function must be called multiple times.
    
    Parameters
        data: Nx1 tensor
            An input to the neural network.
        prediction: Nx1 tensor
            An output of the neural network.
        max_order: float
            Specifies the order up to which derivatives should be calculated.

    Returns
        derivs: Nxmax_order tensor
            No column with the zeroth derivative (the prediction).
    '''
    
    # First derivative builds off prediction.
    derivs = auto.grad(prediction, data, grad_outputs=torch.ones_like(prediction), create_graph=True)[0]
    for _ in range(max_order-1):
        # Higher derivatives chain derivatives from first derivative.
        derivs = torch.cat((derivs, auto.grad(derivs[:, -1:], data, grad_outputs=torch.ones_like(prediction), create_graph=True)[0]), dim=1)
            
    return derivs

## Iteration through tests

In [ ]:
for param_val_1 in param_vals_1:
    
    # Update investigated parameter
    #### Here include lines to update varied attributes. ####
    print('Starting loop with parameter value:', param_val_1)
    noise_level = param_val_1
    
    # Reset randomised initialisation
    np_seed = 2
    torch_seed = 0
    np.random.seed(np_seed)
    torch.manual_seed(torch_seed)

    
    # DATA GENERATION
    input_type = 'Strain'
    
    # Define model
    mech_model = 'GMM'
    E = 3*[1]
    eta = [2.5, 0.5]
    
    # Define manipulation
    func_desc = 'Sinc'
    omega = 1
    amp = 7
    input_expr = lambda t: amp*np.sin(omega*t)/(omega*t)
    d_input_expr = lambda t: (amp/t)*(np.cos(omega*t) - np.sin(omega*t)/(omega*t))
    input_torch_lambda = lambda t: amp*torch.sin(omega*t)/(omega*t)
    
    dg_info_list = [f'Model: '+mech_model, f'E: {E}', f'eta: {eta}', 'Input: '+input_type, 'Desc: '+func_desc, f'omega: {omega}', f'Amp: {amp}']
    
    # Define time series
    time_array = np.linspace(10**-10, 10*np.pi/omega, 5000).reshape(-1, 1)
    
    # Calculate viscoelastic data series
    manipulation_array = input_expr(time_array)
    response_array = VE_datagen.calculate_strain_stress(input_type, time_array, input_expr, d_input_expr, E, eta)
    strain_array, stress_array = (manipulation_array, response_array) if input_type == 'Strain' else (response_array, manipulation_array)
    
    print('Data generated.')
    
    # DATA TREATMENT
    # Scaling
    time_sf = omega/1.2
    strain_sf = 1/np.max(abs(strain_array))
    stress_sf = 1/np.max(abs(stress_array))

    scaled_time_array = time_array*time_sf
    scaled_strain_array = strain_array*strain_sf
    scaled_stress_array = stress_array*stress_sf
    if input_type == 'Strain':
        scaled_input_torch_lambda = lambda t: strain_sf*input_torch_lambda(t/time_sf)
        scaled_target_array = scaled_stress_array
    elif input_type == 'Stress':
        scaled_input_torch_lambda = lambda t: stress_sf*input_torch_lambda(t/time_sf)
        scaled_target_array = scaled_strain_array
    
    # Noise
#     noise_level = 0 # This is the varied parameter in this example.
    noisy_target_array = scaled_target_array + noise_level * np.std(scaled_target_array) * np.random.standard_normal(scaled_target_array.shape)

    # Random sampling
    number_of_samples = 1000
    reordered_row_indices = np.random.permutation(scaled_time_array.size)
    reduced_time_array = scaled_time_array[reordered_row_indices, :][:number_of_samples]
    reduced_target_array = noisy_target_array[reordered_row_indices, :][:number_of_samples]

    
    # CONFIGURATION OF DEEPMOD
    # Conversion to PyTorch tensors
    time_tensor = torch.tensor(reduced_time_array, dtype=torch.float32, requires_grad=True)
    target_tensor = torch.tensor(reduced_target_array, dtype=torch.float32)
    
    # Manipulation variable library advance calculation
    library_diff_order = 3
    input_data = scaled_input_torch_lambda(time_tensor)
    input_derivs = auto_deriv(time_tensor, input_data, library_diff_order)
    input_theta = torch.cat((input_data.detach(), input_derivs.detach()), dim=1)
    
    # Threshold definition
    percent = 0.05
    def thresh_pc(*args): # Keep as full function so that it can be pickled
        return percent
    
    # Definition of configuration dictionaries
    library_config = {'library_func': mech_library,
                      'diff_order': library_diff_order,
                      'coeff_sign': 'positive',
                      'input_type': input_type,
                      'input_theta': input_theta}
    network_config = {'hidden_dim': 30} # Optional
    optim_config = {'lr_coeffs': 0.002,
                    'thresh_func': thresh_pc,
                    'l1': 1e-06} # Optional
    report_config = {'plot': True, 'print_interval': 2000} # Optional
    
    
    # LAUNCHING DEEPMOD
    print('Running DeepMoD...')
    begin_timestamp = datetime.now() # Saves time stamp for beginning of training.
    model = run_deepmod(time_tensor, target_tensor, library_config, network_config, optim_config, report_config) # Extra config dicts not supplied by default.


    # ORGANISING RESULTS
    print('Saving...')
    
    # Prediction arrays
    prediction_tensor = model.network(time_tensor)
    prediction_array = np.array(prediction_tensor.detach())
    time_tensor_post = torch.tensor(scaled_time_array, dtype=torch.float32, requires_grad=True)
    full_prediction_tensor = model.network(time_tensor_post)
    full_prediction_array = np.array(full_prediction_tensor.detach())
    
    # Expected coeffs
    unscaled_coeffs = VE_params.coeffs_from_model_params(E, eta, mech_model)
    expected_coeffs = VE_params.scaled_coeffs_from_true(unscaled_coeffs, time_sf, strain_sf, stress_sf)
    
    # Coeffs arrays
    sparse_coeff_vector_list_list = model.fit.coeff_vector_history
    sparsity_mask_list_list = model.fit.sparsity_mask_history
    
    target_coeffs_array = np.array(expected_coeffs).reshape(-1,1)
    pre_thresh_coeffs_array = np.array(sparse_coeff_vector_list_list[0][0].detach())
    final_coeffs_array = np.array(sparse_coeff_vector_list_list[-1][0].detach())
    sparsity_mask_array = np.array(sparsity_mask_list_list[-1][0]).reshape(-1,1)
    unscaled_final_coeffs = VE_params.true_coeffs_from_scaled(final_coeffs_array, time_sf, strain_sf, stress_sf, mask=sparsity_mask_array, library_diff_order=library_diff_order)
    true_coeffs_array = np.array(unscaled_final_coeffs).reshape(-1, 1)

    # Group like data vectors together for saving
    dg_series_data = np.concatenate((time_array, strain_array, stress_array), axis=1)
    NN_series_data = np.concatenate((reduced_time_array, reduced_target_array, prediction_array), axis=1)
    final_coeffs_data = np.concatenate((final_coeffs_array, true_coeffs_array, sparsity_mask_array), axis=1)
    
    # remove input library from dictionary
    library_config.pop('input_theta', None)
    
    # Gather miscellaneous information into lists for saving
    treatment_info_list = [f'noise_factor: {noise_level}', f'time_sf: {time_sf}', f'strain_sf: {strain_sf}', f'stress_sf: {stress_sf}']
    config_dict_list = [f'library: {library_config}', f'network: {network_config}', f'optim: {optim_config}', f'report: {report_config}']
    dt_string = begin_timestamp.strftime('%d/%m/%Y %H:%M:%S')
    misc_list = ['date_stamp: '+dt_string, f'NumPy_seed: {np_seed}', f'Torch_seed: {torch_seed}']
    
    
    # SAVING RESULTS
    # Collect parameters to name folder for saving
    parent_folder = '../data/Results/Synthetic/Scans/'
    subfolder = f'{investigation_title}/{param_val_1}/'.replace('.', '-')
    foldername = parent_folder + subfolder

    # Make folder
    if not os.path.isdir(foldername):
        os.makedirs(foldername)

    # Save all array data
    np.savetxt(foldername+'/DG_series_data.csv', dg_series_data, delimiter=',', header='Time, Strain, Stress')
    np.savetxt(foldername+'/NN_series_data.csv', NN_series_data, delimiter=',', header='Time, Target, Prediction')
    np.savetxt(foldername+'/expected_coeffs.csv', target_coeffs_array, delimiter=',', header='Expected_coeffs')
    np.savetxt(foldername+'/pre_thresh_coeffs_data.csv', pre_thresh_coeffs_array, delimiter=',', header='Trained_Coeffs')
    np.savetxt(foldername+'/final_coeffs_data.csv', final_coeffs_data, delimiter=',', header='Trained_Coeffs, Unscaled, Sparsity_Mask')
    np.savetxt(foldername+'/full_prediction.csv', full_prediction_array, delimiter=',', header='Full Prediction')
    
    # Save all lists data
    with open(foldername+'/DG_info_list.txt', 'w') as file:
        file.writelines(f'{line}\n' for line in dg_info_list)
        
    with open(foldername+'/treatment_info_list.txt', 'w') as file:
        file.writelines(f'{line}\n' for line in treatment_info_list)
    
    with open(foldername+'/config_dict_list.txt', 'w') as file:
        file.writelines(f'{line}\n' for line in config_dict_list)
    
    with open(foldername+'/misc_list.txt', 'w') as file:
        file.writelines(f'{line}\n' for line in misc_list)
    
    # Save model
    with open(foldername+'/model.pickle', 'wb') as file:
        pickle.dump(model, file) # Will fail on dump if using lambda funcs, will fail on load if normal funcs that are not redefined.